# Lab 6.2: WindowingDemo

## Tổng quan bài tập
**Đề bài**: Hãy hoàn thiện các phần `[...]` để hoàn thiện đoạn code và giải quyết bài toán theo yêu cầu.

## Tài nguyên tham khảo

Bạn có thể tải tập Dataset tại [link sau](https://drive.google.com/file/d/1q0TEA4sEPzv2ghAohGIQQrNdp6JdHFDc/view?usp=sharing). Sau đó đưa lên Google Drive và kết nối với Colab là có thể sử dụng được. Tập dữ liệu là file .parquet có cấu trúc như sau:
```
root
 |-- Country: string (nullable = true)
 |-- WeekNumber: integer (nullable = true)
 |-- NumInvoices: long (nullable = true)
 |-- TotalQuantity: long (nullable = true)
 |-- InvoiceValue: double (nullable = true)

```



# Cài đặt Spark trên Google Colab

Để có thể sử dụng Spark trên môi trường Google Colab thì bạn sẽ cần cài đặt một số thành phần sau:
- Java 8
- Spark Binary
- findspark

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,678 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,333 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://ppa.launchpadcontent.net/c2d4u.team/c2

Sau đó, bạn sẽ cần khai báo cho hệ thống các đường dẫn cho các thành phần vừa cài.

In [3]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

findspark.init()

# Kết nối với Google Drive

Để lấy dữ liệu từ các Dataset, bạn sẽ phải lưu file dữ liệu lên Google Drive. Sau đó kết nối Colab đến Google Drive của bạn và lấy được các file dữ liệu.

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# Average Friends by Age

Bạn sẽ cần khởi tạo 1 SparkSesson để có thể bắt đầu Spark.

In [4]:
from pyspark import SparkConf
from pyspark.sql import SparkSession, Window
from pyspark.sql import functions as f

conf = SparkConf() \
    .setMaster('local') \
    .setAppName('WindowingDemo')

spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext

DATASET_PATH = '/content/gdrive/My Drive/DEP303/summary.parquet'

Đọc dữ liệu từ Dataset

In [5]:
summary_df = spark.read \
    .format("parquet") \
    .option("header","true") \
    .load(DATASET_PATH)
summary_df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- WeekNumber: integer (nullable = true)
 |-- NumInvoices: long (nullable = true)
 |-- TotalQuantity: long (nullable = true)
 |-- InvoiceValue: double (nullable = true)



Hãy sử dụng các thao tác Windowing để tạo thêm cột `RunningTotal` và tính tổng `Invoice` từ tuần đầu tiên đến tuần hiện tại theo các `Country`
<img src = "https://firebasestorage.googleapis.com/v0/b/funix-way.appspot.com/o/xSeries%2FData%20Engineer%2FDEP303x%2FSumary_Image%2FDEP303_sum_L8_4.png?alt=media&token=d9ce255c-42f5-4bf5-9cc3-fde26db5f2bc">

In [6]:
running_total_window = Window.partitionBy("Country") \
    .orderBy("WeekNumber") \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

summary_df.withColumn("RunningTotal",f.sum("InvoiceValue").over(running_total_window)) \
    .show()

+---------------+----------+-----------+-------------+------------+------------------+
|        Country|WeekNumber|NumInvoices|TotalQuantity|InvoiceValue|      RunningTotal|
+---------------+----------+-----------+-------------+------------+------------------+
|      Australia|        48|          1|          107|      358.25|            358.25|
|      Australia|        49|          1|          214|       258.9|            617.15|
|      Australia|        50|          2|          133|      387.95|1005.0999999999999|
|        Austria|        50|          2|            3|      257.04|            257.04|
|        Bahrain|        51|          1|           54|      205.74|            205.74|
|        Belgium|        48|          1|          528|       346.1|             346.1|
|        Belgium|        50|          2|          285|      625.16|            971.26|
|        Belgium|        51|          2|          942|      838.65|1809.9099999999999|
|Channel Islands|        49|          1|   